# ETL - Base de Estornos

###### Data: 21/11/2024

#### Importação da bibliotecas

In [1]:
import pandas as pd
import glob

#### Procurar arquivo no diretório de input

In [2]:
# All files and directories ending with .txt and that don't begin with a dot:
arquivos = glob.glob("./input/*.csv")
print(arquivos) 

['./input\\amostras_input.csv']


#### Definindo os nomes das colunas do df

In [3]:
colunas = ['MES', 'DATA', 'SITE', 'TIPO_COMISSAO', 'DETALHE_COMISSAO', 'ACESSO',
       'PLANO', 'PACOTE', 'TIPO_MIGRACAO', 'SEGMENTO_CRC', 'TIPO_ESTORNO',
       'LOGIN_ATIVACAO', 'ACAO_2', 'PLANO_EVENTO', 'DATA_EVENTO',
       'VALOR_ESTORNO', 'CANAL_N4', 'CLASSIFICACAO']

### Importação do arquivo e tratamento das colunas

In [4]:
# Leitura do arquivo CSV, onde 'arquivos[0]' é o caminho do primeiro arquivo na lista
df = pd.read_csv(arquivos[0], sep=';')  # O arquivo é lido com o delimitador ';'

# Definindo os nomes das colunas do DataFrame, substituindo a atribuição padrão
df.columns = colunas  # 'colunas' é uma lista com os nomes que você quer dar às colunas

# Convertendo a coluna 'VALOR_ESTORNO' para float, substituindo a vírgula por ponto para garantir a conversão correta
df["VALOR_ESTORNO"] = df["VALOR_ESTORNO"].str.replace(',', '.').astype(float)

# Convertendo a coluna 'MES' para formato datetime, considerando que o dia vem antes do mês no formato (dd/mm/yyyy)
df['MES'] = pd.to_datetime(df['MES'], dayfirst=True)

# Convertendo a coluna 'DATA' para formato datetime, com a mesma regra do 'MES'
df['DATA'] = pd.to_datetime(df['DATA'], dayfirst=True)

# Convertendo a coluna 'DATA_EVENTO' para formato datetime, também considerando o dia primeiro
df['DATA_EVENTO'] = pd.to_datetime(df['DATA_EVENTO'], dayfirst=True)

# Aplicando a regex para extrair a parte da string antes do primeiro '__' na coluna 'ACAO_2', e criando uma nova coluna 'ACAO_2__'
df['ACAO_2__'] = df['ACAO_2'].str.extract(r'^([^\s_]+(?:[\w\s-]*))(?=__)')

# Exibindo as três primeiras linhas do DataFrame para verificar o resultado
df.head(3)

,MES,DATA,SITE,TIPO_COMISSAO,DETALHE_COMISSAO,ACESSO,PLANO,PACOTE,TIPO_MIGRACAO,SEGMENTO_CRC,TIPO_ESTORNO,LOGIN_ATIVACAO,ACAO_2,PLANO_EVENTO,DATA_EVENTO,VALOR_ESTORNO,CANAL_N4,CLASSIFICACAO,ACAO_2__
0,2024-06-01,2024-06-01,UPSTREAM,Migração Outbound Pré - Controle,REN - OUTBOUND - Canvass Migrou Ganhou Pré-Con...,91983077209,TIM MAIS CONTROLE A EXPRESS,NaN,PRÉ-CTRL,OUTBOUND,OK,NÃO INFORMADO,AUT_BLOQ_EXP__MAIS_CTRL_A_06_01,NaN,NaT,0.0,CANAIS AUTOMATICOS - OUTBOUND,INCENTIVO,AUT_BLOQ_EXP
1,2024-06-01,2024-06-01,UPSTREAM,Migração Outbound Pré - Controle,REN - OUTBOUND - Canvass Migrou Ganhou Pré-Con...,96981418951,TIM MAIS CONTROLE A EXPRESS,NaN,PRÉ-CTRL,OUTBOUND,OK,NÃO INFORMADO,AUT_BLOQ_EXP__MAIS_CTRL_A_06_01,NaN,NaT,0.0,CANAIS AUTOMATICOS - OUTBOUND,INCENTIVO,AUT_BLOQ_EXP
2,2024-06-01,2024-06-01,UPSTREAM,Migração Outbound Pré - Controle,REN - OUTBOUND - Canvass Migrou Ganhou Pré-Con...,99981857396,TIM MAIS CONTROLE A EXPRESS,NaN,PRÉ-CTRL,OUTBOUND,OK,NÃO INFORMADO,AUT_BLOQ_EXP__MAIS_CTRL_A_06_01,NaN,NaT,0.0,CANAIS AUTOMATICOS - OUTBOUND,INCENTIVO,AUT_BLOQ_EXP


#### Explicação da Regex:
<pre></pre>
^ — Início da string.<br>
([^\s_]+(?:[\w\s-]*)) — Um grupo de captura que:<br>
    - [^\s_]+ — Captura a primeira sequência de caracteres que não sejam espaço ou sublinhado.<br>
    - (?:[\w\s-]*) — Captura uma sequência adicional de caracteres alfanuméricos, espaços ou hífens (usando um grupo não-capturante (?:...) para combinar essas partes sem interromper a captura principal).<br>

(?=__) — Lookahead que garante que o próximo elemento seja o duplo sublinhado __.
</pre>
##### Essa regex agora deve ser capaz de capturar até o primeiro __ de strings mais complexas.

[Um link pra testar RegEx](https://regex101.com/)


#### Check do Tratamento na coluna ACAO_2

In [5]:
acoes_2_tratadas = df['ACAO_2__'].unique()
df_filtro = df[df['ACAO_2__'].isna()]
acoes_2_nao_tratadas = df_filtro['ACAO_2'].unique()
qtde_registros_sem_tratamento = len(df_filtro)
mensagem = """TRATAMENTO CAMPO ACAO_2:
  - {} Ações tratadas com sucesso;
  - {} Ações não tratadas;
As ações não tratadas somam {} registros no dataframe.
""".format(len(acoes_2_tratadas), len(acoes_2_nao_tratadas), qtde_registros_sem_tratamento)
print(mensagem)

TRATAMENTO CAMPO ACAO_2:
  - 83 Ações tratadas com sucesso;
  - 1 Ações não tratadas;
As ações não tratadas somam 371 registros no dataframe.



In [6]:
df_filtro.sample(3)

,MES,DATA,SITE,TIPO_COMISSAO,DETALHE_COMISSAO,ACESSO,PLANO,PACOTE,TIPO_MIGRACAO,SEGMENTO_CRC,TIPO_ESTORNO,LOGIN_ATIVACAO,ACAO_2,PLANO_EVENTO,DATA_EVENTO,VALOR_ESTORNO,CANAL_N4,CLASSIFICACAO,ACAO_2__
2370,2024-06-01,2024-06-01,LIQ AC,Migração Inbound Pré - Controle,REN - INBOUND - Comissionamento Migração Pré-C...,85991605294,TIM CONTROLE LIGACOES ILIMITADAS 6 0,NaN,PRÉ-CTRL,INBOUND,OK,T3693350,_,NaN,NaT,0.0,INBOUND CROSS SELLING,COMISSÃO,NaN
529,2024-06-01,2024-06-02,AEC,Migração Inbound Pré - Controle,REN - INBOUND - Comissionamento Migração Pré-C...,11982213320,TIM CONTROLE LIGACOES ILIMITADAS 6 0,NaN,PRÉ-CTRL,INBOUND,OK,T3703122,_,NaN,NaT,0.0,INBOUND CROSS SELLING,COMISSÃO,NaN
600,2024-06-01,2024-06-02,AEC,Migração Inbound Pré - Controle,REN - INBOUND - Comissionamento Migração Pré-C...,42999733000,TIM CONTROLE LIGACOES ILIMITADAS 6 0,NaN,PRÉ-CTRL,INBOUND,OK,T3713593,_,NaN,NaT,0.0,INBOUND CROSS SELLING,COMISSÃO,NaN


#### Caso o tratamento tenha funcionado adequadamente


In [7]:
# Apagar o campo antigo
del df['ACAO_2']
# Renomear o novo
df.rename(columns={'ACAO_2__': 'ACAO_2'}, inplace=True)
# Reorganiza a ordem natural das colunas
df = df[colunas]
df.sample(3)

,MES,DATA,SITE,TIPO_COMISSAO,DETALHE_COMISSAO,ACESSO,PLANO,PACOTE,TIPO_MIGRACAO,SEGMENTO_CRC,TIPO_ESTORNO,LOGIN_ATIVACAO,ACAO_2,PLANO_EVENTO,DATA_EVENTO,VALOR_ESTORNO,CANAL_N4,CLASSIFICACAO
539,2024-06-01,2024-06-02,AEC,Migração Inbound Pré - Controle,REN - INBOUND - Comissionamento Migração Pré-C...,21981416216,TIM CONTROLE LIGACOES ILIMITADAS 6 0,NaN,PRÉ-CTRL,INBOUND,OK,T3717368,PRE_CTRL_REPROVADOS_,NaN,NaT,0.0,INBOUND CROSS SELLING,COMISSÃO
558,2024-06-01,2024-06-03,LIQ AC,Migração Inbound Pré - Controle,REN - INBOUND - Comissionamento Migração Pré-C...,42998226916,TIM CONTROLE LIGACOES ILIMITADAS 6 0,NaN,PRÉ-CTRL,INBOUND,OK,T3627072,AUT_BLOQ_EXP,NaN,NaT,0.0,INBOUND CROSS SELLING,COMISSÃO
1759,2024-06-01,2024-06-04,ELO,Migração Outbound Controle - Puro,REN - OUTBOUND - Canvass Meta Migração Control...,21969026411,TIM BLACK A LIGHT 6 0,NaN,CTRL-PÓS,OUTBOUND,OK,T3651934,CTRL SMART FIDEL_POS BTL A FIDEL,NaN,NaT,0.0,OUTBOUND,INCENTIVO


##### Exprtação do dataframe em csv

In [8]:
df.to_csv(path_or_buf='./output\\amostra_output.csv', encoding= '1252', sep=';', index=False)

In [9]:
del(df)
del(df_filtro)